# 01 - Automated Testing

The pynhm automated testing is the basis for continuous integration (CI). Coupled with good coverage, CI allows for peace of mind and more rapid and robust code development. The tests themselves also provide a window into how to use the code base in many cases. 

However, the main reason to start with testing as the first notebook (after establishing the pynhm environment) is that the test data are used as input to many of the examples that will follow. This notebook gives a quick overview of generating the test data and running the pynhm tests but does not go into detail on the contents of the tests. 

Automated testing is typically performed on the command line as shown here (though it could be done from within python) and this notebook is to be run in the bash kernel in the pyws_nb conda environment installed in notebook 00.

The automated testing uses pytest. Pytest is an executable called from the command line which has many of its own options. Bear in mind that you can see the full listing of options by typing `pytest --help`. I will highlight several of these here that we will use below.


```
pytest --help

...
  --pdb                 start the interactive Python debugger on errors or KeyboardInterrupt.
  
...
  
  --capture=method      per-test capturing method: one of fd|sys|no|tee-sys.
  -s                    shortcut for --capture=no.
  
...
  
  -v, --verbose         increase verbosity.  
  
...

  -n numprocesses, --numprocesses=numprocesses
                        Shortcut for '--dist=load --tx=NUM*popen'. With 'auto', attempt to detect physical CPU
                        count. With 'logical', detect logical CPU count. If physical CPU count cannot be found,
                        falls back to logical count. This will be 0 when used with --pdb.
```

Pytest generally likes to suppress output to the terminal and keep reporting to a minimum. The assumption is that typically every tests passes. It will report what tests fail at the end of the test and those can be run individually with terminal output (`-s`), increased pytest verbosity (`-v`) and even interactive debugging (`--pdb`). The option to parallelize the tests it helpful as it can dramatically reduce wait time (`-n=auto` or `-n=4`). 

## Requirements: pyws_nb virtual env
The pynhm virtual environment was installed in notebook 00. You need this environment to proceed. __This notebook is to be run with a python kernel using the conda env: pyws_nb.__ This means we'll pass python variables to bash cell magics below, but that seemed to be the most portable solution (on Windows).


## pynhm_root variable
Define the location of the pynhm repository. This should be the location you defined in notebook 00. 

In [ ]:
%load_ext jupyter_black

In [ ]:
import pywatershed
import os
import shutil
import subprocess
import pathlib as pl

# set up paths for notebook
# this is as pywatershed is installed
pynhm_repo_root = pywatershed.constants.__pywatershed_root__.parent
pynhm_test_data_dir = pynhm_repo_root / "test_data"
pynhm_test_data_scripts_dir = pynhm_repo_root / "test_data/scripts"
pynhm_autotest_dir = pynhm_repo_root / "autotest"

In [ ]:
def run_subprocess(args=None, cwd=None, check=False, print_output=True):
    """Helper function for system commands."""
    process = subprocess.run(
        args,
        check=check,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )
    if print_output:
        print(process.stdout.decode("utf8"))
    return process

## Run PRMS to generate test answers and pynhm inputs

By default, the "tests" which run PRMS to generate answers and inputs for pynhm run for all 3 test domains, unless otherwise specified. One can actually see options (specific to this conftest.py) in `pytest --help` output, under "custom options" as will be shown later in this notebook (for the pynhm tests).

The three test domains have their basic data in these folders:

In [ ]:
for domain in ["hru_1", "drb_2yr", "ucb_2yr"]:
    print(f"\n\nDirectory: {domain}")
    dirname = pynhm_test_data_dir / domain
    for entry in dirname.iterdir():
        if entry.is_file():
            print(entry.name, end="  ")

If your repository is not freshly cloned, the above results may not look the same as other files have already been generated (as we will generate below).

Note that on Windows, symbolic links (symlinks) generally require administrator access to set up. We don't want to have to maintain multiple copies of the same file, so as a workaround, we'll copy the files that *are* updated (from 'common') to the domain folders.

In [ ]:
# copy the master 'single_hru' control file to control.test
for domain in ["hru_1"]:
    src = pynhm_test_data_dir / "common/control.single_hru"
    dst = pynhm_test_data_dir / f"{domain}/control.test"
    try:
        shutil.copy(src, dst)
    except shutil.SameFileError:
        pass

# copy the master 'multi_hru' control file to control.test
for domain in ["drb_2yr", "ucb_2yr"]:
    src = pynhm_test_data_dir / "common/control.multi_hru"
    dst = pynhm_test_data_dir / f"{domain}/control.test"
    try:
        shutil.copy(src, dst)
    except shutil.SameFileError:
        pass

The files listed above in each domain directory represent the data needed to run PRMS in an NHM configuration on each of the domains for 2 years in the case of the Delaware River and the Upper Colorado Basins. The inputs for hru_1 allow a 40 year run on a single HRU. More details about these domains will be provided in subsequent notebooks. 

Now we will run PRMS for each of these domains and generate output in an `output/` subdirectory of each domain directory listed above. 

In [ ]:
args = [
    "pytest",
    "-n=4",
    "test_run_domains.py",
]
_ = run_subprocess(
    args=args,
    cwd=pynhm_test_data_scripts_dir,
)

## Convert PRMS outputs to netcdf

PRMS generates CSV output files. For example, for the DRB the file listing is:

In [ ]:
from pprint import pprint as pp

dirname = pynhm_test_data_dir / "drb_2yr/output"
filelist = []
for entry in dirname.iterdir():
    if entry.is_file():
        filelist.append(entry.name)
pp(filelist, compact=True)
print(f"\n Number of files: {len(filelist)}")

We convert these files to netcdf and generate a hand full of extra, derivative files as well in the next step.

In [ ]:
args = [
    "pytest",
    "-n=4",
    "test_nc_domains.py",
]
_ = run_subprocess(
    args=args,
    cwd=pynhm_test_data_scripts_dir,
)

These netcdf files are the results of running PRMS 5.2.1. These files are used for evaluating the results/simulations of pynhm and also as inputs to individual process models (e.g. PRMSRunoff) in pywatershed. Netcdf files can be inspected on the command line with the ncdump utility. Though it's installed with the pyws_nb environment, specifying the path to ncdump is a pain here. Instead, we'll display the datasets from xarray, which is very similar to `ncdump -h`. In the highlevel metadata shown, note that the time durations and number of HRUs are evident by looking at the surface runoff variable (sroff) for each domain. 

In [ ]:
import xarray as xr

for domain in ["hru_1", "drb_2yr", "ucb_2yr"]:
    print(domain)
    display(
        xr.open_dataset(
            pl.Path(f"{pynhm_repo_root}/test_data/{domain}/output/sroff.nc")
        )
    )
    print()

## pynhm autotest
Now we can run the suite of pynhm tests, as we just genereated all the answers and input data. This verifies that your pynhm code base and your virtual environment are copacetic (assuming the commit being tested passed CI). First, I will point out that `pytest --help` even returns options for the test in the current directory under "custom options":

In [ ]:
for ll in stout.splitlines():
    if cc > 7:
        cc = 0
    if "Custom" in str(ll) or (cc > 0 and cc < 7):
        cc += 1
        print(ll.decode("utf-8"))

Now we'll run all pynhm tests.

In [ ]:
args = ["pytest", "-n=4", "--all_domains"]
_ = run_subprocess(args, cwd=pynhm_autotest_dir)

We see that some tests are marked "x" for "expected failure". Some of these fail (x) and some pass (X) as the expected failures are typically just for one of the three domains. We also see generated warnings and the time taken. 